# Kuramoto on sphere
$$ d X_t^i = \left( (A - I \gamma^2) X_t^i + (I- X_t^i (X_t^i)^T) Z  \right) dt + (I- X_t^i (X_t^i)^T) \gamma dW_t^i \quad i=1,2,...,M$$

where $Z = \sum_{i=1}^M \alpha X_t^i + \beta (1,1,1) \quad \beta  \sim Bernoulli(p)$ and $X_t^i \in \mathbb{R}^3$ and $\alpha=\frac{0.5}{M}$, $\beta=\pm 20$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import Algorithms_for_EDMD

In [4]:
def generate_points_on_Sphere(p):
    '''
    Generate random point on unit sphere.
    '''
    X0=np.zeros((3,p))
    d=3
    for j in range(p):
        x = np.random.randn(d)
        y=x / np.linalg.norm(x)
        X0[:,j]=y

    return X0 

def randAntisymmetric(d=3):
    '''
    Generate random antisymmetric matrix.
    '''
    A = np.random.rand(d, d)
    return A - A.T


def generate_A(p):
    d=3
    A = np.zeros((d, d, p))
    for i in range(p):
        A[:, :, i] = randAntisymmetric(d)
    return A


def generate_A_same(p):
    d=3
    A = np.zeros((d, d, p))
    aa=randAntisymmetric(d)
    for i in range(p):
        A[:, :, i] = aa
    return A

def generate_beta(p):
    beta=np.zeros(p)
    unif=np.random.rand(p)
    for j in range(p):
        if unif[j]>=0.5:
            beta[j]=1
        elif unif[j]<0.5:
            beta[j]=-1
    return beta

P = lambda x: np.eye(3) - np.outer(x, x)

In [3]:

def Euler_Kuramoto(Dt,n,A,alphas,gamma,p):
    X = np.zeros((3, p, n))
    
    X[:, :, 0] = generate_points_on_Sphere(p)
    
    #%% Euler-Maruyama    
    for i in range(n-1): # time steps
        beta=generate_beta(p)
        for j in range(p): # particles
            z = beta[j]*np.ones((3))*20 + X[:, :, i] @ alphas 
            x = X[:, j, i]
            X[:, j, i+1] = x + Dt*(A[:, :, j] @ x - (gamma**2)*x + P(x) @ z) + np.sqrt(Dt)*gamma*P(x) @ np.random.randn(3)
            
        for j in range(p):
            X[:, j, i+1] = X[:, j, i+1]/np.linalg.norm(X[:, j, i+1])
            
    
    return X


def Euler_decoupled_Kuramoto(Dt,n,A,alphas,gamma,mu,p):
    X = np.zeros((3, p, n))
    
    X[:, :, 0] = generate_points_on_Sphere(p)
    
    #%% Euler-Maruyama    
    for i in range(n-1): # time steps
        beta=generate_beta(p)
        for j in range(p): # particles
            z = beta[j]*np.ones((3))*20 + mu[:,:,i] @ alphas 
            x = X[:, j, i]
            X[:, j, i+1] = x + Dt*(A[:, :, j] @ x - (gamma**2)*x + P(x) @ z) + np.sqrt(Dt)*gamma*P(x) @ np.random.randn(3)
            
        for j in range(p):
            X[:, j, i+1] = X[:, j, i+1]/np.linalg.norm(X[:, j, i+1])
            
    
    return X


In [ ]:
# Set parameters
p=25000
d=3
T=3
Dt=0.01
t = np.arange(Dt, T + Dt, Dt)
n = len(t) 
X = np.zeros((3, p, n))
gamma = 0.5 
A=generate_A_same(p)
alphas = (0.5/p)*np.ones(p)

# Create data
mu=Euler_Kuramoto(Dt,n,A,alphas,gamma,p)
X=Euler_decoupled_Kuramoto(Dt,n,A,alphas,gamma,mu,p)


In [ ]:
dataX = X[:,:,0]
dataY = X[:,:,-1]

dataY=np.reshape(dataY, (3,p))
dataX=np.reshape(dataX, (3,p)) 


Algorithms_for_EDMD.find_eigenfunctions_eigenvalues_sphere_3D(dataX,dataY,oper='P',num_evs=3,num_boxes=200,plot_efunctions=True)